In [1]:
import math
import random
from random import sample
import numpy as np

def euc_2d(c1,c2):
    return math.sqrt((c1[0]-c2[0])**2+(c1[1]-c2[1])**2)

def cost(sol,cities):
    distance = 0 
    for i,c1 in enumerate(sol):
        if (i == len(sol)-1):
            c2 = sol[0]
        else:
            c2 = sol[i+1]
        distance += euc_2d(cities[c1],cities[c2])
    return distance

def random_permutation(cities):
    perm = np.random.permutation(len(cities))
    return perm

In [6]:
def stochastic_two_opt(permutation):
    perm_size = len(permutation)
    new_perm = []
    x = list(permutation)
    start,end = sorted(sample(range(perm_size),2))
    exclude = [start]
    if start==0:
        exclude.append(perm_size-1)
    else:
        exclude.append(start-1)
    if start==perm_size-1:
        exclude.append(0)
    else:
        exclude.append(start+1)
    while end in exclude:
        end = sample(range(perm_size),1)[0]
    if end < start:
        start,end = end,start
    new_perm = x[:start]
    new_perm += (list(reversed(x[start:end])))
    new_perm += x[end:]
    edges = [[start-1,start],[end-1,end]]
    return new_perm,edges

def is_tabu(x,tabu_list):
    for i,c1 in enumerate(x):
        if i == len(x)-1:
            c2 = x[0]
        else:
            c2 = x[i+1]
        if [c1,c2] in tabu_list:
            return True
    return False

def generate_candidate(best, tabu_list, cities):
    x,edges = None,None
    x,edges = stochastic_two_opt(best['vector'])
    while is_tabu(x,tabu_list):
        x,edges = stochastic_two_opt(best['vector'])
    candidate = {'vector':x}
    candidate['cost'] = cost(candidate['vector'], cities)
    candidate['edges'] = edges
    return candidate

def search(cities, tabu_list_size, candidate_list_size, max_iter):
    current = {'vector': random_permutation(cities)}
    current['cost'] = cost(current['vector'],cities)
    best = current
    tabu_list = []
    for i in range(max_iter):
        candidates = [ generate_candidate(current,tabu_list,cities) for j in range(candidate_list_size)]
        candidates.sort(key = lambda k:k['cost'])
        best_candidate = candidates[0]
        if best_candidate['cost'] < current['cost']:
            current = best_candidate
            if best_candidate['cost'] < best['cost']:
                best = best_candidate
            for  edge in best_candidate['edges']:
                tabu_list.append(edge)
            while len(tabu_list) > tabu_list_size:
                tabu_list.pop()
    return best['vector'],best['cost']

In [7]:
# problem configuration
berlin52 = [[565,575],[25,185],[345,750],[945,685],[845,655],
[880,660],[25,230],[525,1000],[580,1175],[650,1130],[1605,620],
[1220,580],[1465,200],[1530,5],[845,680],[725,370],[145,665],
[415,635],[510,875],[560,365],[300,465],[520,585],[480,415],
[835,625],[975,580],[1215,245],[1320,315],[1250,400],[660,180],
[410,250],[420,555],[575,665],[1150,1160],[700,580],[685,595],
[685,610],[770,610],[795,645],[720,635],[760,650],[475,960],
[95,260],[875,920],[700,500],[555,815],[830,485],[1170,65],
[830,610],[605,625],[595,360],[1340,725],[1740,245]]
# algorithm configuration
max_iter = 1000
tabu_list_size = 50
max_candidates = 500
# execute the algorithm
best = search(berlin52, tabu_list_size, max_candidates, max_iter)

In [8]:
best

([37,
  36,
  38,
  35,
  34,
  33,
  43,
  15,
  49,
  48,
  31,
  30,
  16,
  18,
  40,
  7,
  8,
  9,
  32,
  42,
  3,
  24,
  14,
  4,
  47,
  25,
  26,
  27,
  11,
  50,
  10,
  51,
  13,
  12,
  46,
  28,
  19,
  22,
  29,
  1,
  6,
  41,
  20,
  17,
  2,
  44,
  21,
  0,
  39,
  45,
  23,
  5],
 9611.024324672653)